### Imports

In [ ]:
import numpy as np
import pandas as pd
import ace_tools_open as tools
from case_studies import *
import time
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [3]:
#Save case study functions, their derivatives and hessians in lists
fs = [f1, f4]
dfs = [df1, df4]
Hfs = [Hf1, Hf4]
fnames = ["f1", "f4"]

### scipy BFGS

In [ ]:
def scipy_bfgs(f,df,x0,max_iterations,epsilon):
    xs=[]
    grad_norms=[]
    def logging_f(x):
        xs.append(x)
        grad_norms.append(np.maximum(np.linalg.norm(df(x)),10**(-5)*epsilon))
        return f(x)
    minimize(logging_f, x0, method="BFGS", jac=df, tol=epsilon,options={'maxiter':max_iterations, 'gtol':epsilon})
    return np.array(xs), np.array(grad_norms)

### Strict Wolfe condition Linesearch

In [1]:
def wolfe_search(f, df, x, p, c1, c2, alpha_init):
    def gf(alpha):
        return f(x + np.dot(alpha, p))
    def gdf(alpha):
        return df(x + np.dot(alpha, p))
    l = 0
    u = alpha_init
    while True:
        if gf(u) > gf(0) + c1 * u * gdf(0) or gf(u) > gf(l):
            break
        if abs(gdf(u)) < c2 * abs(gdf(0)):
            alpha_star = u
            return alpha_star
        if gdf(u) > 0:
            break
        else:
            u = u * 2
    while True:
        a = (l + u) / 2
        if gf(a) > gf(0) + c1 * a * gdf(0) or gf(a) > gf(l):
            u = a
        else:
            if abs(gdf(a)) < c2 * abs(gdf(0)):
                alpha_star = a
                return alpha_star
            if gdf(a) < 0:
                l = a
            else:
                u = a

### BFGS

In [ ]:
def bfgs(x, f, df, c1, c2, max_iter):
    # eye tager dimension
    H = np.eye(20)
    for i in range(max_iter):
        p = -np.dot(H, df(x))
        alpha = wolfe_search(f, df, x, p, 1.0, c1, c2)
        ap = np.dot(alpha, p)
        x_new = x + ap
        y = df(x_new) - df(x)
        s = ap
        H = H + 

        x = x_new